In [1]:
from Models import * 
from pathlib import Path
#import re
#import numpy as np 
import io
import os
import pandas as pd

In [27]:
current_directiry =  Path().absolute()

database.connect()



In [ ]:
database.create_tables([Sample, Variants])

In [28]:
def read_vcf(path):
    with open(path, 'r') as f:
        lines = [l for l in f if not l.startswith('##')]
    return pd.read_csv(
        io.StringIO(''.join(lines)),
        dtype={'#CHROM': str, 'POS': int, 'ID': str, 'REF': str, 'ALT': str,
               'QUAL': str, 'FILTER': str, 'INFO': str},
        sep='\t'
    ).rename(columns={'#CHROM': 'CHROM', 'ID' : 'ID_VARIANT'})

standart_columns = {'CHROM', 'POS','ID_VARIANT','REF', 'ALT', 'QUAL','FILTER', 'INFO', 'FORMAT'}

In [29]:
def get_DP(format_, sample):
    info_DP = list(filter(lambda x: x[0]=="DP", zip(format_.split(":"), sample.split(":"))))
    return info_DP[0][1]

def get_GT(format_, sample, ref, alt):
    info_GT = list(filter(lambda x: x[0]=="GT", zip(format_.split(":"), sample.split(":"))))[0][1].split("/") 
    info_GT  = list(map( lambda x: int(x), info_GT  ))
    nucleotides =  [ref] + alt.split(",")
    genotype = nucleotides[info_GT[0]] + "/" + nucleotides[info_GT[1]]
    return genotype

In [30]:
import time
from datetime import timedelta

start_time = time.monotonic()

for fn in Path(current_directiry).glob("*.vcf"):
     
    sample_type = str(fn).split(".")[1]
     
    print(sample_type)
    data = read_vcf(fn)
    sample_names = list( filter(lambda x: x not in standart_columns, list(data)))
    for s in sample_names:
        data["DP"] = data.apply(lambda x: get_DP(x["FORMAT"], x[s]), axis=1)
        data["GT"] = data.apply(lambda x: get_GT(x["FORMAT"], x[s], x["REF"], x["ALT"]), axis=1)
        sample_citonumber = int(s.split("_")[1])
        try:
            sample_to_base = Sample.create(Sample_name = sample_citonumber, Type = sample_type)
            with database.atomic():
                data_source = data[["CHROM", "POS", "ID_VARIANT", "DP", "GT"]].to_dict(orient='records')
                for data_dict in data_source:
                    Variants.create(sample= sample_to_base,**data_dict)
        except IntegrityError:
            continue
    
end_time = time.monotonic()
print(timedelta(seconds=end_time - start_time))

Panel
Exome
0:00:21.925077


In [31]:
#data
#list_of_dicts = convert_df_to_list_of_dicts(df)

In [32]:
for sample in Sample.select():
    #Sample.get(Sample.Sample_name == sample.Sample_name).delete_instance()
    print(sample.Sample_name, sample.Type)

3002 Exome
3330 Panel


In [34]:
database.close()

True

In [35]:
#for sample in Variants.select():
    #Sample.get(Sample.Sample_name == sample.Sample_name).delete_instance()
    #print(sample.ID_VARIANT, sample.sample)